In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from functools import reduce
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix
import sparse_dot_topn.sparse_dot_topn as ct
import ftfy
import re


In [ ]:
#First three datasets come from https://www.kaggle.com/datasets/stevenpeutz/misinformation-fake-news-text-dataset-79k

In [2]:
FN1 = pd.read_csv('DataSet_Misinfo_FAKE.csv', index_col=0)

In [3]:
FN1

,text
0,Donald Trump just couldn t wish all Americans ...
1,House Intelligence Committee Chairman Devin Nu...
2,"On Friday, it was revealed that former Milwauk..."
3,"On Christmas day, Donald Trump announced that ..."
4,Pope Francis used his annual Christmas Day mes...
...,...
44422,The USA wants to divide Syria.\r\n\r\nGreat Br...
44423,The Ukrainian coup d'etat cost the US nothing ...
44424,The European Parliament falsifies history by d...
44425,The European Parliament falsifies history by d...


In [4]:
FN1.head()

,text
0,Donald Trump just couldn t wish all Americans ...
1,House Intelligence Committee Chairman Devin Nu...
2,"On Friday, it was revealed that former Milwauk..."
3,"On Christmas day, Donald Trump announced that ..."
4,Pope Francis used his annual Christmas Day mes...


In [5]:
FN1.columns

Index(['text'], dtype='object')

In [6]:
#Adding a column to label it as real news or fake news as the datasets themselves were not labeled excpt for filename

In [7]:
FN1['real'] = 0

In [8]:
FN1

,text,real
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
...,...,...
44422,The USA wants to divide Syria.\r\n\r\nGreat Br...,0
44423,The Ukrainian coup d'etat cost the US nothing ...,0
44424,The European Parliament falsifies history by d...,0
44425,The European Parliament falsifies history by d...,0


In [9]:
FN1['text'][0]

'Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev

In [10]:
TN1 = pd.read_csv('DataSet_Misinfo_TRUE.csv', index_col=0)

In [11]:
TN1.head()

,text
0,The head of a conservative Republican faction ...
1,Transgender people will be allowed for the fir...
2,The special counsel investigation of links bet...
3,Trump campaign adviser George Papadopoulos tol...
4,President Donald Trump called on the U.S. Post...


In [12]:
TN1.columns

Index(['text'], dtype='object')

In [13]:
#Again adding a column to the data as it was not labelled as real or fake within dataset

In [14]:
TN1['real'] = 1

In [15]:
TN1

,text,real
0,The head of a conservative Republican faction ...,1
1,Transgender people will be allowed for the fir...,1
2,The special counsel investigation of links bet...,1
3,Trump campaign adviser George Papadopoulos tol...,1
4,President Donald Trump called on the U.S. Post...,1
...,...,...
34970,Most conservatives who oppose marriage equalit...,1
34971,The freshman senator from Georgia quoted scrip...,1
34972,The State Department told the Republican Natio...,1
34973,"ADDIS ABABA, Ethiopia —President Obama convene...",1


In [16]:
TN1['text'][0]

'The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support education, scientific researc

In [17]:
RusN = pd.read_csv('EXTRA_RussianPropagandaSubset.csv', index_col=0)

In [18]:
RusN

,text
0,Ukraine has put itself in a situation when ext...
1,Regardless who was behind the recent attack on...
2,"Pilsudski is a historical figure, who establis..."
3,Washington (and to a large degree Brussels) ar...
4,The beneficiary of the resolution of the Europ...
...,...
7364,The USA wants to divide Syria.\r\n\r\nGreat Br...
7365,The Ukrainian coup d'etat cost the US nothing ...
7366,The European Parliament falsifies history by d...
7367,The European Parliament falsifies history by d...


In [19]:
RusN.columns

Index(['text'], dtype='object')

In [20]:
#Russian propaganda extra dataset from Kaggle, again fake and needs labelled

In [21]:
RusN['real'] = 0

In [22]:
RusN

,text,real
0,Ukraine has put itself in a situation when ext...,0
1,Regardless who was behind the recent attack on...,0
2,"Pilsudski is a historical figure, who establis...",0
3,Washington (and to a large degree Brussels) ar...,0
4,The beneficiary of the resolution of the Europ...,0
...,...,...
7364,The USA wants to divide Syria.\r\n\r\nGreat Br...,0
7365,The Ukrainian coup d'etat cost the US nothing ...,0
7366,The European Parliament falsifies history by d...,0
7367,The European Parliament falsifies history by d...,0


In [23]:
RusN['text'][0]

'Ukraine has put itself in a situation when external forces dictate how it should solve its internal problems. Kyiv expected that its Western allies will put pressure on Russia in the Normandy Four format. However, the allies realised that their confrontation with Russia will cost them more. Emmanuel Macron and Angela Merkel supported Russia\'s position and put even bigger pressure on Ukraine than Vladimir Putin did. Eventually Zelenskyy found himself in solitude, because Russia, Germany, and France had nearly identical positions regarding the resolution of the civil conflict in Ukraine. Ukrainian politicians wrongfully remain confident that the West will always unconditionally support any Russophobic regime and that Ukraine can just carry out its foreign policy and security under the West\'s authority to enjoy its protection.\r\n\r\nUkraine could slam the door and declare that it no longer needs the Normandy format. However, it is the last remaining material evidence of Ukraine\'s uni

In [ ]:
# Remaining datasets come from https://www.kaggle.com/datasets/emineyetm/fake-news-detection-datasets

In [24]:
FN2 = pd.read_csv('FAKE.csv')

In [25]:
FN2

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [26]:
FN2['real'] = 0

In [27]:
FN2

,title,text,subject,date,real
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0
...,...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016",0
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016",0
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016",0
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016",0


In [28]:
FN2.isnull().values.any()

False

In [29]:
#Have to eliminate close duplicates

In [30]:
FN2['text'][0]

'Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev

In [31]:
TN2 = pd.read_csv('TRUE.csv')

In [32]:
TN2.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [33]:
TN2.columns

Index(['title', 'text', 'subject', 'date'], dtype='object')

In [34]:
#This dataset also does not have a label for fake or real news

In [35]:
TN2['real'] = 1

In [36]:
TN2

,title,text,subject,date,real
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1
...,...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017",1
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017",1
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017",1
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017",1


In [37]:
#Concerning that the datasets seem to have to first articles as nearly identical. 

In [38]:
TN2['text'][0]

'WASHINGTON (Reuters) - The head of a conservative Republican faction in the U.S. Congress, who voted this month for a huge expansion of the national debt to pay for tax cuts, called himself a “fiscal conservative” on Sunday and urged budget restraint in 2018. In keeping with a sharp pivot under way among Republicans, U.S. Representative Mark Meadows, speaking on CBS’ “Face the Nation,” drew a hard line on federal spending, which lawmakers are bracing to do battle over in January. When they return from the holidays on Wednesday, lawmakers will begin trying to pass a federal budget in a fight likely to be linked to other issues, such as immigration policy, even as the November congressional election campaigns approach in which Republicans will seek to keep control of Congress. President Donald Trump and his Republicans want a big budget increase in military spending, while Democrats also want proportional increases for non-defense “discretionary” spending on programs that support educat

In [39]:
#Combining the first three datasets as they have similar columns, method was chosen for less lines of code

In [40]:
News_df = reduce(lambda  left,right: pd.merge(left,right,on=['text', 'real'],
                                            how='outer'), [FN1, TN1, RusN])


In [41]:
News_df.columns

Index(['text', 'real'], dtype='object')

In [42]:
#This checks the first three datasets for empty values. Surprisingly there are some. I will eliminate them

In [43]:
News_df.isnull().values.any()

True

In [44]:
null_counts = News_df.isnull().sum()
null_counts[null_counts > 0].sort_values(ascending=False)

text    32
dtype: int64

In [45]:
News_df.dropna(inplace = True)
News_df

,text,real
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0
...,...,...
85403,It is not the border between Norway and Russia...,0
85404,NATO soldiers in uniforms are scaring away Rus...,0
85405,Russia agrees with China's stance on US and We...,0
85406,\r\n\r\nTallinn is fighting furiously with its...,0


In [46]:
News_df.isnull().values.any()

False

In [47]:
#Inserting these columns for ease of the reduce method of joining dataframes. 

In [48]:
News_df['subject'] = np.nan
News_df['title'] = np.nan
News_df['date'] = np.nan

In [49]:
News_df

,text,real,subject,title,date
0,Donald Trump just couldn t wish all Americans ...,0,NaN,NaN,NaN
1,House Intelligence Committee Chairman Devin Nu...,0,NaN,NaN,NaN
2,"On Friday, it was revealed that former Milwauk...",0,NaN,NaN,NaN
3,"On Christmas day, Donald Trump announced that ...",0,NaN,NaN,NaN
4,Pope Francis used his annual Christmas Day mes...,0,NaN,NaN,NaN
...,...,...,...,...,...
85403,It is not the border between Norway and Russia...,0,NaN,NaN,NaN
85404,NATO soldiers in uniforms are scaring away Rus...,0,NaN,NaN,NaN
85405,Russia agrees with China's stance on US and We...,0,NaN,NaN,NaN
85406,\r\n\r\nTallinn is fighting furiously with its...,0,NaN,NaN,NaN


In [50]:
News_df.isnull().values.any()

True

In [51]:
News_df.isnull()

,text,real,subject,title,date
0,False,False,True,True,True
1,False,False,True,True,True
2,False,False,True,True,True
3,False,False,True,True,True
4,False,False,True,True,True
...,...,...,...,...,...
85403,False,False,True,True,True
85404,False,False,True,True,True
85405,False,False,True,True,True
85406,False,False,True,True,True


In [52]:
News_df

,text,real,subject,title,date
0,Donald Trump just couldn t wish all Americans ...,0,NaN,NaN,NaN
1,House Intelligence Committee Chairman Devin Nu...,0,NaN,NaN,NaN
2,"On Friday, it was revealed that former Milwauk...",0,NaN,NaN,NaN
3,"On Christmas day, Donald Trump announced that ...",0,NaN,NaN,NaN
4,Pope Francis used his annual Christmas Day mes...,0,NaN,NaN,NaN
...,...,...,...,...,...
85403,It is not the border between Norway and Russia...,0,NaN,NaN,NaN
85404,NATO soldiers in uniforms are scaring away Rus...,0,NaN,NaN,NaN
85405,Russia agrees with China's stance on US and We...,0,NaN,NaN,NaN
85406,\r\n\r\nTallinn is fighting furiously with its...,0,NaN,NaN,NaN


In [53]:
News_df = News_df.fillna('None')

In [54]:
News_df

,text,real,subject,title,date
0,Donald Trump just couldn t wish all Americans ...,0,None,None,None
1,House Intelligence Committee Chairman Devin Nu...,0,None,None,None
2,"On Friday, it was revealed that former Milwauk...",0,None,None,None
3,"On Christmas day, Donald Trump announced that ...",0,None,None,None
4,Pope Francis used his annual Christmas Day mes...,0,None,None,None
...,...,...,...,...,...
85403,It is not the border between Norway and Russia...,0,None,None,None
85404,NATO soldiers in uniforms are scaring away Rus...,0,None,None,None
85405,Russia agrees with China's stance on US and We...,0,None,None,None
85406,\r\n\r\nTallinn is fighting furiously with its...,0,None,None,None


In [55]:
#Combining all the data

In [56]:
News_df = reduce(lambda  left,right: pd.merge(left,right,on=['text', 'real', 'subject', 'title', 'date'],
                                            how='outer'), [News_df, TN2, FN2])

In [57]:
News_df

,text,real,subject,title,date
0,Donald Trump just couldn t wish all Americans ...,0,None,None,None
1,House Intelligence Committee Chairman Devin Nu...,0,None,None,None
2,"On Friday, it was revealed that former Milwauk...",0,None,None,None
3,"On Christmas day, Donald Trump announced that ...",0,None,None,None
4,Pope Francis used his annual Christmas Day mes...,0,None,None,None
...,...,...,...,...,...
130269,21st Century Wire says As 21WIRE reported earl...,0,Middle-east,McPain: John McCain Furious That Iran Treated ...,"January 16, 2016"
130270,21st Century Wire says It s a familiar theme. ...,0,Middle-east,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,"January 16, 2016"
130271,Patrick Henningsen 21st Century WireRemember ...,0,Middle-east,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,"January 15, 2016"
130272,21st Century Wire says Al Jazeera America will...,0,Middle-east,How to Blow $700 Million: Al Jazeera America F...,"January 14, 2016"


In [58]:
#Obviously there are duplicates, what might be a good cut off to eliminate the duplicates. Also are the datasets basically the
#same? Might be a limitation on Fake News training in general. 
#Partial was chosen as topics may be similar and reword certain ideas, but different news sources. 

In [59]:
#fuzz.partial_ratio(FN2['text'][0],FN1['text'][0])

In [60]:
130274/2

65137.0

In [61]:
x = (len(News_df['text'])-1)

In [62]:
#Traditional fuzzy matching was taking too long and 
#I did not need to compare every letter to itself.
#Also I limited number of characters and increased chunksize of text for increased efficiency with little theoretical downside. 
#Thus Tdidf was already implemented in the finding of near duplicates
#code adapted from https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536 and
#https://bergvca.github.io/2017/10/14/super-fast-string-matching.html
#citation for ftfy: Robyn Speer. (2021). ftfy (Version 6.0).


In [63]:
def ngrams(string, n = 3):
    string = ftfy.fix_text(string)
    string = string.encode('ascii', errors='ignore').decode()
    string = string.lower()
    chars_to_remove = [')',')','.','|','[',']','{','}',"'"]
    rx = '[' + re.escape(''.join(chars_to_remove)) + ']'
    string = re.sub(rx, '', string)
    string = string.replace('&', 'and')
    string = string.replace(',',' ')
    string = string.replace('-', ' ')
    string = string.title()
    string = re.sub(' +',' ',string).strip()
    string = ' '+ string +' '
    #string = re.sub(r'[,-./]|\sBD',r'', string) (test this later)
    ngrams = zip(*[string[i:40] for i in range (n)])
    return [''.join(ngram) for ngram in ngrams]

In [64]:
News_arts = News_df['text'].unique()
vectorizer = TfidfVectorizer(min_df=1, analyzer = ngrams)
tf_idf_matrix = vectorizer.fit_transform(News_arts)

In [65]:
#print(tf_idf_matrix)

In [66]:
def awesome_cossim_top(A, B, ntop, lower_bound=0):
    A = A.tocsr()
    B = B.tocsr()
    M, _ = A.shape
    _, N = B.shape
 
    idx_dtype = np.int32
 
    nnz_max = M*ntop
 
    indptr = np.zeros(M+1, dtype=idx_dtype)
    indices = np.zeros(nnz_max, dtype=idx_dtype)
    data = np.zeros(nnz_max, dtype=A.dtype)
    ct.sparse_dot_topn(
        M, N, np.asarray(A.indptr, dtype=idx_dtype),
        np.asarray(A.indices, dtype=idx_dtype),
        A.data,
        np.asarray(B.indptr, dtype=idx_dtype),
        np.asarray(B.indices, dtype=idx_dtype),
        B.data,
        ntop,
        lower_bound,
        indptr, indices, data)
    return csr_matrix((data,indices,indptr),shape=(M,N))

In [67]:
matches = awesome_cossim_top(tf_idf_matrix, tf_idf_matrix.transpose(),1000,.9)


In [68]:
def get_matches_df(sparse_matrix, name_vector, top=100):
    non_zeros = sparse_matrix.nonzero()
    
    sparserows = non_zeros[0]
    sparsecols = non_zeros[1]
    
    if top:
        nr_matches = top
    else:
        nr_matches = sparsecols.size
    
    left_side = np.empty([nr_matches], dtype=object)
    right_side = np.empty([nr_matches], dtype=object)
    similairity = np.zeros(nr_matches)
    
    for index in range(0, nr_matches):
        left_side[index] = name_vector[sparserows[index]]
        right_side[index] = name_vector[sparsecols[index]]
        similairity[index] = sparse_matrix.data[index]
    
    return pd.DataFrame({'left_side': left_side,
                          'right_side': right_side,
                           'similairity': similairity})

In [69]:
#Nothing below 1.0 seems to be an exact match, lets look at it sorted

In [70]:
matches_df = get_matches_df(matches, News_df['text'], top=1000)
matches_df = matches_df[matches_df['similairity'] < 0.99999]
matches_df.sample(20)

,left_side,right_side,similairity
249,White House Chief of Staff John Kelly is a Fou...,They call and promise (to join the European Un...,0.911822
246,White House Chief of Staff John Kelly is a Fou...,Countering Hillary’s Coup With a Counter Coup ...,0.978085
6,House Intelligence Committee Chairman Devin Nu...,"The private investigator, Robert Mueller, had ...",0.952191
889,Donald Trump s conflicts of interest are final...,Donald Trump s conflicts of interest were of m...,0.951444
72,Former reality show star Donald Trump just can...,Former reality show star Donald Trump has alwa...,0.943566
248,White House Chief of Staff John Kelly is a Fou...,Abby Martin Exposes What Hillary Clinton Reall...,0.939115
800,Former reality show star Donald Trump launched...,(Reuters) - Six U.S. Senate Democrats on Thurs...,0.929603
685,Donald Trump s conflicts of interest were of m...,Donald Trump s conflicts of interest are final...,0.951444
374,Republicans are desperately trying to repeal O...,Republicans are desperately trying to convince...,0.963788
85,Former reality show star Donald Trump has repe...,(Reuters) - Six U.S. Senate Democrats on Thurs...,0.943566


In [71]:
#It seems that we can reasonably assume that less than 1.0 are not duplicates

In [72]:
matches_df.sort_values(['similairity'], ascending=False).head(20)


,left_side,right_side,similairity
540,Milwaukee County Sheriff David Clarke seems li...,NATO intends to turn the entire country of Mon...,0.989772
539,Milwaukee County Sheriff David Clarke seems li...,Brussels and Washington accuse Russia of inter...,0.989772
247,White House Chief of Staff John Kelly is a Fou...,"A Birmingham, Alabama ABC affiliate sought out...",0.978085
241,White House Chief of Staff John Kelly is a Fou...,DUBAI (Reuters) - Iran said on Wednesday it ha...,0.978085
246,White House Chief of Staff John Kelly is a Fou...,Countering Hillary’s Coup With a Counter Coup ...,0.978085
245,White House Chief of Staff John Kelly is a Fou...,-Onions -Red peppers \nNotice they are all hea...,0.978085
244,White House Chief of Staff John Kelly is a Fou...,By LAURA MOWAT \nLocals are fuming because the...,0.978085
243,White House Chief of Staff John Kelly is a Fou...,BNI Store Nov 6 2016 Like a good little sharia...,0.978085
242,White House Chief of Staff John Kelly is a Fou...,The U.S. Justice Department on Monday said it ...,0.978085
5,House Intelligence Committee Chairman Devin Nu...,"If this keeps up, the Democrats will never win...",0.972830


In [73]:
#Granted some of the below do not seem to be exact duplicates. 
#I am inclined for simplicity sake to remove all with a similarity of 1.0, but lets look at these first 5
#There are 226 'exact matches' much less than I was fearful of at the beginning
#But I have to remember that this is a sample of the dataset. And that this is potentially 25% of my data.
# Lets run the original 

In [74]:
matches_df = get_matches_df(matches, News_df['text'], top=1000)
matches_df = matches_df[matches_df['similairity'] ==1.0]
matches_df.sample(20)

,left_side,right_side,similairity
153,Tuesday was a big election night for an off-ye...,Tuesday was a big election night for an off-ye...,1.0
35,Donald Trump held a rally for Alabama Senate c...,Donald Trump held a rally for Alabama Senate c...,1.0
139,It didn t take long for conservative Christia...,It didn t take long for conservative Christia...,1.0
397,Trump s desperation to convince people that th...,Trump s desperation to convince people that th...,1.0
359,John McCain is far more qualified to comment o...,John McCain is far more qualified to comment o...,1.0
418,By now you would be more than aware of Preside...,By now you would be more than aware of Preside...,1.0
405,Unless you ve been living under a rock since J...,Unless you ve been living under a rock for the...,1.0
865,Donald Trump s approval rating continues to pl...,Donald Trump s approval rating continues to pl...,1.0
849,"You thought the people passing through Fargo, ...","You thought the people passing through Fargo, ...",1.0
727,The current occupant of the White House has be...,The current occupant of the White House is occ...,1.0


In [75]:
matches_df.head(10)

,left_side,right_side,similairity
0,Donald Trump just couldn t wish all Americans ...,Donald Trump just couldn t wish all Americans ...,1.0
8,"On Friday, it was revealed that former Milwauk...","On Friday, it was revealed that former Milwauk...",1.0
11,The number of cases of cops brutalizing and ki...,The number of cases of cops brutalizing and ki...,1.0
17,Republicans are working overtime trying to sel...,Republicans are working overtime trying to sel...,1.0
21,Donald Trump just signed the GOP tax scam into...,Donald Trump just signed the GOP tax scam into...,1.0
25,Senate Majority Whip John Cornyn (R-TX) though...,Senate Majority Whip John Cornyn (R-TX) though...,1.0
35,Donald Trump held a rally for Alabama Senate c...,Donald Trump held a rally for Alabama Senate c...,1.0
37,"In America, we have been having a conversation...","In America, we have been having a conversation...",1.0
38,New questions are being asked about President ...,New questions are being asked about President ...,1.0
39,"On Wednesday, Donald Trump took a step no prev...","On Wednesday, Donald Trump took a step no prev...",1.0


In [76]:
matches_df.tail(10)

,left_side,right_side,similairity
953,A new report released by the Congressional Bud...,A new report released by the Congressional Bud...,1.0
961,The contrast between the Obama White House and...,The contrast between the Obama White House and...,1.0
966,A new report has just revealed that Fox News p...,A new report has just revealed that Fox News p...,1.0
973,"Secretary of State Rex Tillerson, America s to...","Secretary of State Rex Tillerson, America s to...",1.0
977,"For months now, the Republicans have been doin...","For months now, the Republicans have been doin...",1.0
985,"Donald Trump, at 71 years old, apparently just...","Donald Trump, at 71 years old, apparently just...",1.0
989,As the Russia stories surrounding the criminal...,As the Russia stories surrounding the criminal...,1.0
993,"By now, everyone is used to the rank ignorance...","By now, everyone is used to the rank ignorance...",1.0
995,It turns out that Donald Trump was abusing his...,It turns out that Donald Trump was abusing his...,1.0
998,Keeping in line with his constant backtracking...,Keeping in line with his constant backtracking...,1.0


In [77]:
#It seems reasonable to assume that anything identified as similarity of 1.0 is indeed a duplicate. 
#Now to remove them. 
#I have to be careful as these were a sample of my datasets. 
#Let's confirm I don't have more than 8000 duplicates

In [78]:
matches_df = get_matches_df(matches, News_df['text'], top=8000)
matches_df = matches_df[matches_df['similairity'] ==1.0]
matches_df

,left_side,right_side,similairity
0,Donald Trump just couldn t wish all Americans ...,Donald Trump just couldn t wish all Americans ...,1.0
8,"On Friday, it was revealed that former Milwauk...","On Friday, it was revealed that former Milwauk...",1.0
11,The number of cases of cops brutalizing and ki...,The number of cases of cops brutalizing and ki...,1.0
17,Republicans are working overtime trying to sel...,Republicans are working overtime trying to sel...,1.0
21,Donald Trump just signed the GOP tax scam into...,Donald Trump just signed the GOP tax scam into...,1.0
...,...,...,...
7980,Republicans are desperately trying to convince...,Republicans are desperately trying to convince...,1.0
7989,It is literally impossible for Donald Trump to...,It is literally impossible for Donald Trump to...,1.0
7992,Donald Trump loves to brag that he is the stro...,Donald Trump loves to brag that he is the stro...,1.0
7995,It s no secret that Trump s campaign isn t exa...,It s no secret that Trump s campaign isn t exa...,1.0


In [79]:
#2278 out of 130274 are duplicates, let's check percentage

In [80]:
2278/130274

0.01748622134884935

In [81]:
#Less than 2 percent, that is perfectly reasonable given that we are working with a few data sets. 
#Let's remove these from News_df

In [82]:
matches_df['left_side'][0]

'Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout out to his enemies, haters and  the very dishonest fake news media.  The former reality show star had just one job to do and he couldn t do it. As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year,  President Angry Pants tweeted.  2018 will be a great year for America! As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America!  Donald J. Trump (@realDonaldTrump) December 31, 2017Trump s tweet went down about as welll as you d expect.What kind of president sends a New Year s greeting like this despicable, petty, infantile gibberish? Only Trump! His lack of decency won t ev

In [83]:
# just checking to make sure that matches_df retained the entire article

In [84]:
x = len(News_df['text'])
x

130274

In [85]:
130274-2278
y = 2278

In [86]:
#The above should be the approximate size of the resulting dataframe
#Choosing only one side of the matches has a good chance of only deleting one option. 
#as long as News_df is reasonably close to 127996, I will consider it a success.

In [92]:
matches_df = matches_df.reset_index()

    

In [93]:
matches_df

,index,left_side,right_side,similairity
0,0,Donald Trump just couldn t wish all Americans ...,Donald Trump just couldn t wish all Americans ...,1.0
1,8,"On Friday, it was revealed that former Milwauk...","On Friday, it was revealed that former Milwauk...",1.0
2,11,The number of cases of cops brutalizing and ki...,The number of cases of cops brutalizing and ki...,1.0
3,17,Republicans are working overtime trying to sel...,Republicans are working overtime trying to sel...,1.0
4,21,Donald Trump just signed the GOP tax scam into...,Donald Trump just signed the GOP tax scam into...,1.0
...,...,...,...,...
2273,7980,Republicans are desperately trying to convince...,Republicans are desperately trying to convince...,1.0
2274,7989,It is literally impossible for Donald Trump to...,It is literally impossible for Donald Trump to...,1.0
2275,7992,Donald Trump loves to brag that he is the stro...,Donald Trump loves to brag that he is the stro...,1.0
2276,7995,It s no secret that Trump s campaign isn t exa...,It s no secret that Trump s campaign isn t exa...,1.0


In [ ]:
dup_indices = []
for i in range(x):
    for j in range(y):
        if News_df['text'][i] == matches_df['left_side'][j]:
            News_df.drop([i])
print(News_df)